# download and uzip glove model

In [ ]:
# prompt: download glove model from this url : https://huggingface.co/stanfordnlp/glove/resolve/main/glove.42B.300d.zip

!wget https://huggingface.co/stanfordnlp/glove/resolve/main/glove.42B.300d.zip


--2024-06-02 16:05:10--  https://huggingface.co/stanfordnlp/glove/resolve/main/glove.42B.300d.zip
Resolving huggingface.co (huggingface.co)... 3.163.189.74, 3.163.189.114, 3.163.189.90, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/stanfordnlp/glove/357baac33090f645e71e253b3295ee1b767c98a0336e9a1d99c77e9e33b43c4a?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27glove.42B.300d.zip%3B+filename%3D%22glove.42B.300d.zip%22%3B&response-content-type=application%2Fzip&Expires=1717603510&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzYwMzUxMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9zdGFuZm9yZG5scC9nbG92ZS8zNTdiYWFjMzMwOTBmNjQ1ZTcxZTI1M2IzMjk1ZWUxYjc2N2M5OGEwMzM2ZTlhMWQ5OWM3N2U5ZTMzYjQzYzRhP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiZyZXNwb25zZS1jb250ZW50LXR5cGU9KiJ9XX0_&Signature=fgwaFujjnU

In [ ]:
# prompt: unzip this medel to my drive in folder IR

!unzip -j glove.42B.300d.zip -d drive/MyDrive/IR


Archive:  glove.42B.300d.zip
  inflating: drive/MyDrive/IR/glove.42B.300d.txt  


# Load Docs from drive

In [4]:
# prompt: read docs from my drive

import json

with open('/content/drive/MyDrive/IR/docs_tokenized.json', 'r') as f:
    docs = json.load(f)

print(docs[0])

{'id': 1, 'doc_id': 'NCT00884858', 'text': ['maraviroc', 'immunological', 'nonresponder', 'inr', 'hiv1infected', 'subjectsuse', 'maraviroc', 'mvc', 'immunological', 'nonresponder', 'hiv1infected', 'patient', 'hiv', 'infection', 'objective', 'evaluate', 'clinical', 'efficacy', 'haart', 'intensification', 'mvc', 'treatment', 'hiv1', 'infection', 'patient', 'cd4', 'count', '≤', '200', 'cellsul', 'andor', 'recovery', 'cd4', 'cell', '25', 'compare', 'haart', 'initiation', 'complete', 'stable', 'virologic', 'suppression', '12', 'month', 'haart', 'patient', 'could', 'also', 'include', 'cd4', 'slope', 'stable', 'without', 'improvement', 'absolute', 'value', 'around', '200', 'cellsul', 'evaluate', 'effect', 'haart', 'intensification', 'mvc', 'modification', 'immunologic', 'virologic', 'parameter', 'evaluate', 'tolerability', 'haart', 'intensification', 'mvc', 'appearance', 'drugrelated', 'side', 'effect', 'design', 'randomise', 'multicenter', 'study', 'evaluate', 'haart', 'intensification', 'mv

# Load Glove Embeddings from txt file

In [5]:
import numpy as np

def load_glove_embeddings(glove_file_path):
    emmbed_dict = {}
    with open(glove_file_path,'r') as f:
      for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:],'float32')
        emmbed_dict[word]=vector

    return emmbed_dict

# Load the GloVe embeddings 
glove_file_path = '/content/drive/MyDrive/IR/vectors.txt'
glove_embeddings = load_glove_embeddings(glove_file_path)



# save Glove embeddings to drive

In [6]:
# prompt: save glove_embeddings to my drive

import pickle

with open('/content/drive/MyDrive/IR/my_glove_embeddings.pkl', 'wb') as f:
  pickle.dump(glove_embeddings, f)


# Load Glove embeddings from drive

In [ ]:
# prompt: Load Glove embeddings from drive
import pickle
with open('/content/drive/MyDrive/IR/glove_embeddings.pkl', 'rb') as f:
    glove_embeddings = pickle.load(f)


# Document Vectors

In [7]:
import numpy as np
def document_to_vector(doc, embeddings):
    vector = np.zeros(300)  # Assuming 300-dimensional GloVe vectors
    count = 0
    for word in doc:
        if word in embeddings:
            vector += embeddings[word]
            count += 1
    if count > 0:
        vector /= count
    return vector


In [8]:
document_vectors = [document_to_vector(doc['text'], glove_embeddings) for doc in docs]

print(len(document_vectors[0]))
print(len(document_vectors))

300
241006


# save documrnt vectors to drive

In [9]:
# prompt: save document_vectors to my drive

import pickle

with open('/content/drive/MyDrive/IR/my_document_vectors.pkl', 'wb') as f:
  pickle.dump(document_vectors, f)


# Load document vectors from drive

In [ ]:
# prompt: # Load document vectors from drive
import pickle
with open('/content/drive/MyDrive/IR/document_vectors.pkl', 'rb') as f:
    document_vectors = pickle.load(f)


# Vector Stor

## install pinecone

In [10]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 1.8 MB/s eta 0:00:00


## init index

In [11]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key='e07a8850-6879-43dd-8087-c4841a61d2fd')

index_name = "clinicaltrailscustomgloveindex"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=300,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

index= pc.Index(index_name)

## upsert vectors

In [12]:
batch_size = 1000  # Size of each batch
total_docs = len(docs)  # Total number of documents

print(total_docs)
print(len(document_vectors))

batch = []

for i, vector in enumerate(document_vectors):
    doc = docs[i]
    vector = {
        "id": doc['doc_id'],
        "values": vector
    }
    batch.append(vector)

    # Check if the batch size is reached or if it's the last document
    if (i + 1) % batch_size == 0 or (i + 1) == total_docs:
        index.upsert(
            vectors=batch,
            namespace="clinicaltrails"
        )
        # Clear the batch for the next set of vectors
        batch = []

241006
241006


# Match Query

In [13]:
from nltk.tokenize import word_tokenize
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## test

In [14]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    return tokens

def query_to_vector(query, embeddings):
    tokenized_query = preprocess_text(query)
    return document_to_vector(tokenized_query, embeddings)

query = "Colon cancer KRAS (G13D), BRAF (V600E) 52-year-old male Type II Diabetes, Hypertension"
query_vector = query_to_vector(query, glove_embeddings).tolist()

print(query_vector)
# Retrieve similar documents
results = index.query(vector=query_vector, top_k=10, namespace='clinicaltrails')

print(results)


[0.3002750016748905, -0.34122230038046836, -0.12322939708828926, 0.13662079647183417, -0.0988503996282816, 0.24697830276563765, 0.09793980158865452, 0.15614139810204505, -0.22603209540247918, 0.21577749969437718, -0.08951440081000328, -0.15849149972200394, -0.02476399764418602, -0.23854679577052593, 0.5183481007814408, -0.09848949871957302, 0.05720579167827964, -0.05436569601297379, -0.1176874965429306, 0.02751520350575447, -0.18114831000566484, 0.25872999951243403, -0.06391780190169812, 0.26520999558269975, 0.05780010148882866, 0.25267269313335416, 0.2152088983450085, 0.00046909861266613006, 0.33314330540597437, 0.08807260319590568, -0.31440949328243734, -0.08542720228433609, 0.01978940237313509, 0.1798949989490211, -0.27081289812922477, -0.8020217031240463, 0.7641064956784248, 0.2575596019625664, 0.17539719883352517, 0.04800120145082474, -0.03416319862008095, 0.1428021957864985, -0.043582397326827046, -0.15998269990086555, 0.05569129884243011, 0.4292312994599342, -0.20801679734140635

# Evaluation

In [15]:
# prompt: read cleaned_queries from my drive

import csv

queries = []

# Read the queries from the CSV file
with open('/content/drive/MyDrive/IR/evaluation/cleaned_queries.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        query_id, query = row
        queries.append((query_id, query))

print(queries[0])

('1', 'liposarcoma cdk4 amplification 38yearold male gerd')


In [16]:
def find_similar_documents(query, top_n=10):
    query_vector = query_to_vector(query, glove_embeddings).tolist()
    results = index.query(vector=query_vector, top_k=top_n, namespace='clinicaltrails')
    return results['matches']

In [17]:
results = {}

for query_id, query in queries:
    if int(query_id) == 10:
        continue
    similar_docs = find_similar_documents(query, top_n=10)
    doc_ids = [doc['id'] for doc in similar_docs]
    results[int(query_id)] = doc_ids

print(results)
print(len(results))

{1: ['NCT01121588', 'NCT00410592', 'NCT00561730', 'NCT02187783', 'NCT02137343', 'NCT01697072', 'NCT01774851', 'NCT02731313', 'NCT00688675', 'NCT01220934'], 2: ['NCT02827565', 'NCT01925274', 'NCT01719380', 'NCT02039336', 'NCT01243372', 'NCT01417442', 'NCT01086267', 'NCT01152437', 'NCT02109653', 'NCT02663232'], 3: ['NCT00255671', 'NCT02684058', 'NCT00967200', 'NCT02494622', 'NCT00004483', 'NCT00499798', 'NCT00002647', 'NCT02974127', 'NCT00322790', 'NCT01626846'], 4: ['NCT02440620', 'NCT02214004', 'NCT00897208', 'NCT00958971', 'NCT00133796', 'NCT02924584', 'NCT02302742', 'NCT00582322', 'NCT02202746', 'NCT01251900'], 5: ['NCT02663232', 'NCT01400451', 'NCT02827565', 'NCT01417442', 'NCT01543113', 'NCT00936221', 'NCT01744860', 'NCT02967692', 'NCT01775943', 'NCT02501551'], 6: ['NCT01320085', 'NCT02663232', 'NCT02967692', 'NCT02027961', 'NCT01682083', 'NCT00235482', 'NCT00936221', 'NCT01597908', 'NCT01245556', 'NCT01721772'], 7: ['NCT02961270', 'NCT01775943', 'NCT01774721', 'NCT01106781', 'NCT0

In [18]:
import pickle

with open('/content/drive/MyDrive/IR/evaluation/clinical_trials_qrels.pickle', 'rb') as f:
  qrels = pickle.load(f)

print(qrels.get(1,[]))
print(len(qrels))

[{'doc_id': 'NCT00445783', 'rel': 1}, {'doc_id': 'NCT01209598', 'rel': 2}, {'doc_id': 'NCT01237236', 'rel': 1}, {'doc_id': 'NCT01522989', 'rel': 1}, {'doc_id': 'NCT01692496', 'rel': 2}, {'doc_id': 'NCT02022982', 'rel': 1}, {'doc_id': 'NCT02065063', 'rel': 1}, {'doc_id': 'NCT02187783', 'rel': 1}, {'doc_id': 'NCT02414724', 'rel': 1}, {'doc_id': 'NCT02418234', 'rel': 1}, {'doc_id': 'NCT02571829', 'rel': 2}, {'doc_id': 'NCT02693535', 'rel': 1}, {'doc_id': 'NCT02846987', 'rel': 2}, {'doc_id': 'NCT02897375', 'rel': 1}, {'doc_id': 'NCT02919696', 'rel': 1}, {'doc_id': 'NCT03065062', 'rel': 1}, {'doc_id': 'NCT03096912', 'rel': 2}]
29


## Presision@K


In [19]:
import numpy as np

def calculate_relevant_count(retrieved_docs: list, query_docs: list) -> int:
    """
    Find the number of intersected documents between `retrieved_docs` and `query_docs`

    Args:
        retrieved_docs: Al list of doc_id that returned from matching
        query_docs: A list of doc_id belonging to a qid from qrel file

    Returns:
        Number of shared results
    """
    intersect_values = np.intersect1d(retrieved_docs, query_docs)
    matched_count = len(intersect_values)
    return matched_count


def precision_at_k(relevant_docs, retrieved_docs, k=10) -> float | int:
    """
    Calculates Precision@k

    Args:
        relevant_docs: A dictionary mapping query IDs to a list of relevant document IDs.
        retrieved_docs: A dictionary mapping query IDs to a list of retrieved document IDs, ranked by relevance.
        k: The number of top retrieved documents to consider (default 10).

    Returns:
        The Precision value.
    """

    retrieved = retrieved_docs[:k]
    num_retrieved = len(retrieved)
    num_retrieved_relevant = calculate_relevant_count(retrieved, relevant_docs)
    return num_retrieved_relevant / num_retrieved if num_retrieved > 0 else 0





## Average precision


In [20]:
def get_rel_from_list(list_of_rel, doc_id) -> int:
    """Get the rel for a given doc_id from a list of {'doc_id': 'NCT00445783', 'rel': 1}, ..."""

    for rel in list_of_rel:
        if rel['doc_id'] == doc_id:
            return rel['rel']

    return 0




def average_precision(retrieved: list, relevant: list):
    p_sum = 0
    num_of_relevant = 0
    for i in range(10):
        k = i + 1

        # get the doc_id's for the current
        relevant_docs = [doc['doc_id'] for doc in relevant]

        p_at_k = precision_at_k(relevant_docs, retrieved, k)
        print(f'P@{k} : {p_at_k}')
        # get the k document id
        k_doc_id = retrieved[:k][-1]

        # get the rel(k)
        rel_at_k = get_rel_from_list(relevant, k_doc_id)

        if rel_at_k > 0:
            num_of_relevant += rel_at_k
        p_sum += p_at_k * rel_at_k

    return p_sum / num_of_relevant if num_of_relevant > 0 else 0


# MAP


In [21]:
def mean_average_precision(queries: dict, qrels: dict) -> float | int:
    ap_sum = 0
    for qid, query_results in queries.items():
        if qid == 10:
            continue
        print(f'query number {qid} : ')
        val = average_precision(queries[qid], qrels[qid])
        print(f'******* Average Precision : {val}')
        ap_sum += val
        print('-------------------------------------')
    return ap_sum / len(queries)

mean_average_precision(results, qrels)

query number 1 : 
P@1 : 0.0
P@2 : 0.0
P@3 : 0.0
P@4 : 0.25
P@5 : 0.2
P@6 : 0.16666666666666666
P@7 : 0.14285714285714285
P@8 : 0.125
P@9 : 0.1111111111111111
P@10 : 0.1
******* Average Precision : 0.25
-------------------------------------
query number 2 : 
P@1 : 1.0
P@2 : 1.0
P@3 : 0.6666666666666666
P@4 : 0.5
P@5 : 0.6
P@6 : 0.5
P@7 : 0.5714285714285714
P@8 : 0.625
P@9 : 0.5555555555555556
P@10 : 0.5
******* Average Precision : 0.7096938775510205
-------------------------------------
query number 3 : 
P@1 : 0.0
P@2 : 0.0
P@3 : 0.0
P@4 : 0.0
P@5 : 0.2
P@6 : 0.16666666666666666
P@7 : 0.14285714285714285
P@8 : 0.125
P@9 : 0.1111111111111111
P@10 : 0.1
******* Average Precision : 0.2
-------------------------------------
query number 4 : 
P@1 : 0.0
P@2 : 0.0
P@3 : 0.0
P@4 : 0.0
P@5 : 0.0
P@6 : 0.0
P@7 : 0.14285714285714285
P@8 : 0.125
P@9 : 0.2222222222222222
P@10 : 0.2
******* Average Precision : 0.19576719576719573
-------------------------------------
query number 5 : 
P@1 : 1.0
P@2 :

0.22245614720984183